# Installation librairies

# Import librairies

In [1]:
import pandas as pd
import numpy as np
import requests
import os

# DATA IMPORT:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# chemins
path = '/content/drive/MyDrive/ICU/'
path_down = '/content/drive/MyDrive/ICU/Download_data/'
path_analyse = '/content/drive/MyDrive/ICU/Analyse/'
path_carto = '/content/drive/MyDrive/ICU/Carto/'

# Dataset import

In [4]:
#import delta T°
df_delta = pd.read_csv(path_analyse+'dataset_ICU.csv',sep=';')

In [5]:
# Remplir la colonne 'ICU' avec 1 si 'delta_temperature' est supérieur ou égal à 4, sinon mettre 0
df_delta['ICU'] = df_delta['delta_temperature'].apply(lambda x: 1 if x >= 4 else 0)

# Calcul de la moyenne de 'delta_temperature'
df_delta_mean = df_delta.groupby(['id', 'station_name', 'type_de_station'])['delta_temperature'].mean()

# Comptage des ICU
count_ICU = df_delta[df_delta['ICU'] == 1].groupby(['id', 'station_name', 'type_de_station'])['ICU'].count()

# Combinaison des deux résultats dans un même DataFrame
result = pd.concat([df_delta_mean, count_ICU], axis=1)

In [6]:
#import stations data
df_station = pd.read_csv(path_down+'Liste_station.csv',sep=';')
df_station = df_station[df_station['emission']!='N'][['id_numero','longitude','latitude','altitude','id_nom','lcz','ville','geopoint']]
df_station = df_station.sort_values('id_numero').reset_index(drop=True)

In [7]:
# Lister les tables html
List_tables = [f for f in os.listdir(path_carto) if os.path.isfile(os.path.join(path_carto, f)) and f.endswith('table.html')]

# Fichiers à supprimer de la liste
files_to_remove = ['Station_analyse.html']

# Utilisation d'une boucle pour supprimer les fichiers de la liste s'ils existent
for file in files_to_remove:
    if file in List_tables:
        List_tables.remove(file)

#print(List_tables)

In [8]:
# Fonction pour lire une table depuis un fichier HTML et la convertir en DataFrame
def read_html_table(file_path):
    try:
        tables = pd.read_html(file_path)  # Renvoie une liste de toutes les tables trouvées dans le fichier HTML
        if tables:  # Vérifie s'il y a au moins une table
            return tables[0]  # Si une seule table, on retourne la première
    except ValueError as e:
        print(f"Aucune table trouvée dans {file_path}. Erreur: {e}")
    return None

# Créer une liste pour stocker tous les DataFrames
dataframes = []

# Boucle pour lire chaque fichier HTML et l'ajouter à la liste des DataFrames
for file in List_tables:
    file_path = os.path.join(path_carto, file)
    df = read_html_table(file_path)
    if df is not None:
        dataframes.append(df)

# Concaténer tous les DataFrames en un seul
if dataframes:
    Station_analyse = pd.concat(dataframes, ignore_index=True)

    # Afficher ou enregistrer le DataFrame final
    #display(Station_analyse)

    # Sauvegarder dans un fichier CSV ou HTML si nécessaire
    #Station_analyse.to_csv(os.path.join(path_carto, 'Station_analyse.csv'), index=False)
    #Station_analyse.to_html(os.path.join(path_carto, 'Station_analyse.html'), index=False)
else:
    print("Aucune table HTML n'a été trouvée ou fusionnée.")



In [10]:
# Fusionner les DataFrames df_station & result puis df_ent
df_station_all = pd.merge(Station_analyse, df_station, left_on='Station name', right_on='id_nom', how='left')
df_station_all = pd.merge(df_station_all, result, left_on='id_nom', right_on='station_name', how='left')

# Supprimer les colonnes inutiles
df_station_all = df_station_all.drop(columns=['Station name','Station'])
df_station_all = df_station_all.rename(columns={'Surface BÃ¢timents': 'Surface Batiments',
                                                'Hauteur moyenne des bÃ¢timents': 'Hauteur moyenne des batiments',
                                                'Surface Zones PiÃ©tonnes + Parking': 'Surface Zones Pietonnes + Parking',
                                                'Surface diverses (non categorisÃ©e)': 'Surface diverses (non categorisee)'})
#reorder les colonnes
df_station_all_ordered = df_station_all[['id_numero',	'longitude',	'latitude',	'altitude',	'id_nom',	'lcz',	'ville',	'geopoint',
                                         'delta_temperature',	'ICU',
                                         'Surface Batiments',	'Hauteur moyenne des batiments',	'Surface Zones Humides:',	'Surface Espaces Verts',
                                         'Nombre d\'arbres', 'Surface Zones Pietonnes + Parking',	'Surface diverses (non categorisee)',	'Population_totale_300m',	'Nb entreprises' ]]

# Exporter resultats_population_par_quartier en un fichier CSV
output_path = os.path.join(path_analyse, 'Station_analyse_all.csv')  # Spécifier le chemin de sauvegarde du fichier CSV
df_station_all_ordered.to_csv(output_path, sep=';', index=False)

print(f"DataFrame exporté en CSV à : {output_path}")



DataFrame exporté en CSV à : /content/drive/MyDrive/ICU/Analyse/Station_analyse_all.csv
